#### Calculating unpublished CPI series

March 8, 2018

list of series: https://download.bls.gov/pub/time.series/cu/cu.series



In [1]:
import pandas as pd

In [13]:
series_list_url = 'https://download.bls.gov/pub/time.series/cu/cu.series'
series_list = pd.read_table(series_list_url, sep='\t').set_index('series_id')['series_title']
series_list[series_list.str.contains('food', case=False)]

series_id
CUSR0000SA0L1        All items less food in U.S. city average, all ...
CUSR0000SA0L12       All items less food and shelter in U.S. city a...
CUSR0000SA0L12E      All items less food, shelter, and energy in U....
CUSR0000SA0L12E4     All items less food, shelter, energy, and used...
CUSR0000SA0L1E       All items less food and energy in U.S. city av...
CUSR0000SACL1        Commodities less food in U.S. city average, al...
CUSR0000SACL11       Commodities less food and beverages in U.S. ci...
CUSR0000SACL1E       Commodities less food and energy commodities i...
CUSR0000SACL1E4      Commodities less food, energy, and used cars a...
CUSR0000SAF          Food and beverages in U.S. city average, all u...
CUSR0000SAF1         Food in U.S. city average, all urban consumers...
CUSR0000SAF11        Food at home in U.S. city average, all urban c...
CUSR0000SAF115       Other food at home in U.S. city average, all u...
CUSR0000SAN1D        Domestically produced farm food in U.S. city a

In [53]:
series_list.head()

series_id
CUSR0000SA0          All items in U.S. city average, all urban cons...
CUSR0000SA0E         Energy in U.S. city average, all urban consume...
CUSR0000SA0L1        All items less food in U.S. city average, all ...
CUSR0000SA0L12       All items less food and shelter in U.S. city a...
CUSR0000SA0L12E      All items less food, shelter, and energy in U....
Name: series_title, dtype: object

#### Test case: reconstructing the all-items CPI inflation rate from the all-items minus food and the food indices

In [83]:
100.0 - 13.384

86.616

In [84]:
rel_imp = [('CUUR0000SAF1', 13.891, '2013-12-01', '2015-11-01'), 
           ('CUUR0000SAF1', 14.015, '2015-12-01', '2017-11-01'), 
           ('CUUR0000SAF1', 13.384, '2017-12-01', '2019-11-01'),
           ('CUUR0000SA0L1', 86.109, '2013-12-01', '2015-11-01'), 
           ('CUUR0000SA0L1', 85.985, '2015-12-01', '2017-11-01'), 
           ('CUUR0000SA0L1', 86.616, '2017-12-01', '2019-11-01'),
           ('CUUR0000SA0', 100, '2013-12-01', '2015-11-01'), 
           ('CUUR0000SA0', 100, '2015-12-01', '2017-11-01'), 
           ('CUUR0000SA0', 100, '2017-12-01', '2019-11-01')]

In [89]:
series = ['CUUR0000SAF1', 'CUUR0000SA0L1', 'CUUR0000SA0']
#data_url = 'https://download.bls.gov/pub/time.series/cu/cu.data.0.Current'
#data = pd.read_table(data_url, sep='\s+')
df = data[(data['series_id'].isin(series)) & (data['year'] > 2013) 
          & (data['period'] != 'M13')]
df['date'] = [pd.to_datetime(f"{i['year']}-{i['period'][1:]}-01") for r, i in df.iterrows()]
#df.set_index(['series_id', 'date'])['value'].to_csv('food.csv', header=True)

C:\Users\BDew\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [90]:
df['ri'] = [i[1] for r, d in df.iterrows() for i in rel_imp if d['date'] >= pd.to_datetime(i[2]) 
            and d['date'] <= pd.to_datetime(i[3]) and d['series_id'] == i[0]]

C:\Users\BDew\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
df = df.set_index(['series_id', 'date'])

In [86]:
df.to_csv('food2.csv', index=False, header=True)

In [95]:
1 + df['value'].pct_change().

series_id     date      
CUUR0000SA0   2014-01-01         NaN
              2014-02-01    1.003698
              2014-03-01    1.006440
              2014-04-01    1.003297
              2014-05-01    1.003493
              2014-06-01    1.001862
              2014-07-01    0.999610
              2014-08-01    0.998329
              2014-09-01    1.000753
              2014-10-01    0.997488
              2014-11-01    0.994601
              2014-12-01    0.994330
              2015-01-01    0.995294
              2015-02-01    1.004343
              2015-03-01    1.005952
              2015-04-01    1.002033
              2015-05-01    1.005097
              2015-06-01    1.003503
              2015-07-01    1.000067
              2015-08-01    0.998584
              2015-09-01    0.998443
              2015-10-01    0.999550
              2015-11-01    0.997889
              2015-12-01    0.996583
              2016-01-01    1.001653
              2016-02-01    1.000823
             

In [51]:
df['ri'] = None

In [52]:
df

series_id      date      
CUSR0000SA0L1  2006-01-01      200.3
               2006-02-01      200.3
               2006-03-01      200.7
               2006-04-01      201.9
               2006-05-01      202.6
               2006-06-01        203
               2006-07-01      204.2
               2006-08-01      205.1
               2006-09-01      203.9
               2006-10-01      202.8
               2006-11-01      202.9
               2006-12-01      204.1
               2007-01-01    204.297
               2007-02-01    204.992
               2007-03-01    206.114
               2007-04-01    206.747
               2007-05-01    207.609
               2007-06-01    208.032
               2007-07-01    208.349
               2007-08-01    208.295
               2007-09-01    209.157
               2007-10-01    209.799
               2007-11-01    211.575
               2007-12-01    212.225
               2008-01-01    212.866
               2008-02-01    213.336
            

In [15]:
data_url = 'https://download.bls.gov/pub/time.series/cu/cu.data.0.Current'

In [24]:
data = pd.read_table(data_url, sep='\s+')

In [27]:
df = data[(data['series_id'].isin(series)) & (data['year'] > 2005)]

In [36]:
df['date'] = [f"{i['year']}-{i['period'][1:]}-01" for r, i in df.iterrows()]

C:\Users\BDew\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df.set_index(['series_id', 'date'])['value'].to_csv('test.csv')

In [33]:
for i in df.iterrows():
    print(i)

(614, series_id         CUSR0000SA0L1
year                       2006
period                      M01
value                     200.3
footnote_codes              NaN
Name: 614, dtype: object)
(615, series_id         CUSR0000SA0L1
year                       2006
period                      M02
value                     200.3
footnote_codes              NaN
Name: 615, dtype: object)
(616, series_id         CUSR0000SA0L1
year                       2006
period                      M03
value                     200.7
footnote_codes              NaN
Name: 616, dtype: object)
(617, series_id         CUSR0000SA0L1
year                       2006
period                      M04
value                     201.9
footnote_codes              NaN
Name: 617, dtype: object)
(618, series_id         CUSR0000SA0L1
year                       2006
period                      M05
value                     202.6
footnote_codes              NaN
Name: 618, dtype: object)
(619, series_id         CUSR0000SA0L1
ye

In [29]:
df.to_csv('test.csv', index=False)